In [1]:
%cd ../mmk2

/home/antoine/Gdrive/Autoencoders/mmk2


In [2]:
import torch
import torch.nn as nn
import numpy as np
import math
import os
import sys

sys.path = [p for p in sys.path if "Auto" not in p]

from mmk.freqnet import FreqNet
from mmk.data import Database, download_database
from mmk.kit import get_trainer


# connect to neptune
api_token = os.environ.get("NEPTUNE_API_TOKEN", None)


/home/antoine/anaconda3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [3]:
# get your data
db = download_database(api_token,
                       project_name="k-tonal/data-and-base-notebooks",
                       experiment_id="DAT-1",
                       database_name="gould.h5",
                       destination="../")
# db = Database("../gould.h5")
db

<Database(../gould.h5)>

In [5]:
db.fft

{}

In [ ]:
# build a model and its trainer and fit

fnet = FreqNet(
    # data_object can be an array, a tensor or a db.feature : FreqNet infers its dimensionality!
    data_object=db.fft.get(db.metadata.iloc[[0, 1]]),
    splits=[.85, .15],
     model_dim=256,
     groups=1,
     n_layers=(int(np.log2(64)),),
     strict=False,
     accum_outputs=None,
     concat_outputs=None,
     pad_input=None,
     learn_padding=False,
    input_seq_length=128,
                 )

trainer = get_trainer(model=fnet,
                      max_epochs=10,
                      # epochs controls checkpoints :
                      # if its a int -> checkpoint every x epoch ; if it's a list -> only at [x, y, z, ] epochs
                      epochs=20,
                      # where you will store everything
                      root_dir="../freqnet_test/gould",
                     neptune_api_token=api_token,
                     neptune_project="k-tonal/model-upload-test")

trainer.fit(fnet)

In [ ]:
for i in range(8):
    audio = fnet.generate(fnet.random_train_example(), n_steps=2048, hop_length=db.fft.attrs["hop_length"])
    fnet.log_audio("test_wav_" + str(i), audio)

In [ ]:
!tensorboard --logdir ../freqnet_test/gould/

In [ ]:
fnet.upload_to_neptune()

In [ ]:
import neptune
import os

api_token = os.environ["NEPTUNE_API_TOKEN"]
session = neptune.Session.with_default_backend(api_token=api_token)
project = session.get_project("k-tonal" + "/" + "model-upload-test")
exp = project.get_experiments(id="MOD-32")[0]
exp.delete_artifacts("states")